# Numeric Tests

This notebook runs some numerical tests to assess the validity, accuracy, and performance of numeric python packages for the Weierstrass elliptic functions. Where appropriate it compares two alternative versions. 

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# Original package https://github.com/stla/pyweierstrass/blob/main/pyweierstrass/weierstrass.py
from weierstrass import omega_from_g, g_from_omega, wp, invwp, wpprime

# Modified package
from weierstrass_modified import Weierstrass
wst = Weierstrass()

from random import random
from mpmath import almosteq, mpc, mpf, im, timing, polyroots
from sympy import *

In [46]:
omega1, omega2, omega3, g2, g3, tau, G4, G6, q = symbols('omega1, omega2, omega3, g2, g3, tau, G4, G6, q')

## 1. Calculating the half-periods from the elliptic invariants

- The series for g2 in terms of G4 is quartic in omega1. When solving for omega1 in terms of g2 the result is thus only determined up to a quartic root of unity.
- Because the result for omega1 can thus sometimes be incorrectly multiplied by +1, -1, +i, or -i, we can observe errors when trying to recalculate g2 and g3 from omega and the mistake manifests as giving the wrong sign for g3.
- One solution is to include the series for g3 in terms of G6 to decide whether to multiply the periods by the imaginary unit or not, 
    then the periods are defined up to a sign +/-1. In other words, the ratio g2/g3 is quadratic in the half-periods not quartic. 
- As the half periods are only defined up to a sign because of periodicity anyway this suffices and elliminates the problem of obtaining the wrong sign for g3 when calculating from omega.
- By G4 and G6 we mean G4(1, tau), G6(1, tau) with G_2k(tau) = 2 * zeta(2*k) * E_2k(tau), with zeta the Riemann zeta function and E the Eisenstein series expressible in terms of theta functions

A single example of how the original method for calculating the half-periods can fail is given below as g3 takes the wrong sign

In [95]:
# Input values of g2 and g3 are chosen
g2_num =  mpc(real='-73.4914329314137', imag='95.8413827483826')
g3_num =  mpc(real='54.2026055822565', imag='27.1486316597472')

# The values for g2 and g3 are then calculated from omega1 and omega2 whcih in turn are calculated from the orginal gs g3
# This should return the original values but it returns -g3 instead
g_from_omega(*omega_from_g(g2_num, g3_num))

(mpc(real='-73.491432931413684', imag='95.841382748382628'),
 mpc(real='-54.202605582253817', imag='-27.14863165974365'))

So instead of doing this:

In [55]:
g2_G4_eq = Eq(g2, Rational(60, 2**4) * G4 / omega1**4)
g2_G4_eq

Eq(g2, 15*G4/(4*omega1**4))

In [56]:
Eq(omega1, (Rational(60, 2**4) * G4/ g2) ** (Rational(1, 4)))

Eq(omega1, 15**(1/4)*sqrt(2)*(G4/g2)**(1/4)/2)

It is recommended that we do this:

In [62]:
g2_G4_eq

Eq(g2, 15*G4/(4*omega1**4))

In [57]:
g3_G6_eq = Eq(g3, Rational(140,2**6) * G6 / omega1**6 )
g3_G6_eq

Eq(g3, 35*G6/(16*omega1**6))

In [58]:
Eq(g2_G4_eq.lhs / g3_G6_eq.lhs, g2_G4_eq.rhs / g3_G6_eq.rhs)

Eq(g2/g3, 12*G4*omega1**2/(7*G6))

In [60]:
Eq(omega1, sqrt(Rational(7, 12) * g2 * G6 / (g3 * G4)))

Eq(omega1, sqrt(21)*sqrt(G6*g2/(G4*g3))/6)

Calulating half-periods from g2 and g3, then calculating g2 and g3 from the half-periods, then comparing the calculated values to the orignal g2 and g3 values.

In [94]:
def test_accuracy_of_g_from_omega_and_omega_from_g(
    omega_from_g_function,
    g_from_omega_function,
    Ntests = 1000,
    num_span = 100,
    tolerance = 1e-10
):

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define random values for g2 and g3 
            # Picks values from within the complex square -num_span ... num_span, -1i*num_span ... 1i*num_span
            g2_num = mpc(real=f'{num_span*2*(random() - 0.5)}', imag=f'{num_span*2*(random() - 0.5)}')
            g3_num = mpc(real=f'{num_span*2*(random() - 0.5)}', imag=f'{num_span*2*(random() - 0.5)}')

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega1 = omegas[0]
            omega2 = omegas[1]

            # Swap the sign of omega2 if needed to get an allowed value for tau = omega2/omega1
            sign_flipped = False
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
                sign_flipped = True

            # Calculate g2 and g3 from the half-periods
            g2calc, g3calc = g_from_omega_function(omega1, omega2)

            # Compare the accuracy of the calculated value of g2 to the original
            err_1 = False
            if not almosteq(g2_num, g2calc, tolerance):
                err_msg_1 = f'g2calc={g2calc} not within tolerance for g2_num={g2_num}'
                err_1 = True

            # Compare the accuracy of the calculated value of g3 to the original
            err_2 = False
            if not almosteq(g3_num, g3calc, tolerance):
                err_msg_2 = f'g3calc={g3calc} not within tolerance for g3_num={g3_num}'
                err_2 =True

            # Raise an exception if any calculated values for g2 and g3 
            # differ from the original values by more than the tolerance
            err_msg = ""
            if err_1:
                err_msg += err_msg_1 + '\n'
            if err_2:
                err_msg += err_msg_2 + '\n'
            if err_1 or err_2:
                raise Exception(err_msg)

        except Exception as e:

            # Print the values and error messages for which accuracy was outside the tolerance
            print('Error:\n', e)
            print('g2',g2_num)
            print('g3',g3_num)
            print('sign_flipped', sign_flipped)

            # Track the error rate 
            err_count += 1

    # Print the final error rate
    err_rate = err_count / Ntests
    print(err_rate)

In [89]:
# Using the original package this shows the problems that occur with incorrect half-periods
# leading to g3 taking the wrong sign approximately half the time
test_accuracy_of_g_from_omega_and_omega_from_g(
    omega_from_g,
    g_from_omega
)

Error:
 g3calc=(-83.7363346989625 + 22.5430369888079j) not within tolerance for g3_num=(83.7363346989629 - 22.5430369888073j)

g2 (8.4594578853715 + 55.8556613958734j)
g3 (83.7363346989629 - 22.5430369888073j)
sign_flipped False
Error:
 g3calc=(53.4398387721798 - 55.7156581178248j) not within tolerance for g3_num=(-53.4398387721807 + 55.7156581178238j)

g2 (49.269501644038 - 38.3226019229218j)
g3 (-53.4398387721807 + 55.7156581178238j)
sign_flipped False
Error:
 g3calc=(-26.1317478979266 - 38.7509397513098j) not within tolerance for g3_num=(26.1317478979269 + 38.7509397513101j)

g2 (33.1404172753007 - 53.9413671171519j)
g3 (26.1317478979269 + 38.7509397513101j)
sign_flipped False
Error:
 g3calc=(-53.0243624385199 + 95.5723649983485j) not within tolerance for g3_num=(53.024362438522 - 95.5723649983479j)

g2 (-80.8404369590583 - 20.2358081973057j)
g3 (53.024362438522 - 95.5723649983479j)
sign_flipped False
Error:
 g3calc=(-73.2508649201562 + 51.9796859197232j) not within tolerance for g3

Error:
 g3calc=(80.491095322849 + 92.8957206528641j) not within tolerance for g3_num=(-80.491095322849 - 92.8957206528644j)

g2 (-16.33331295133 - 26.4628043795326j)
g3 (-80.491095322849 - 92.8957206528644j)
sign_flipped False
Error:
 g3calc=(30.9406013163003 + 55.4119393956848j) not within tolerance for g3_num=(-30.9406013163012 - 55.4119393956852j)

g2 (30.9099324269042 + 98.9516385912311j)
g3 (-30.9406013163012 - 55.4119393956852j)
sign_flipped False
Error:
 g3calc=(95.8057836920264 - 66.6389838743705j) not within tolerance for g3_num=(-95.8057836920287 + 66.638983874372j)

g2 (-97.7056536035117 + 39.2686780143909j)
g3 (-95.8057836920287 + 66.638983874372j)
sign_flipped False
Error:
 g3calc=(-4.79636385323078 - 84.6956753926063j) not within tolerance for g3_num=(4.79636385322884 + 84.6956753926077j)

g2 (-74.0032793288023 + 51.7759746328241j)
g3 (4.79636385322884 + 84.6956753926077j)
sign_flipped False
Error:
 g3calc=(-26.7638701117204 - 33.9657930719654j) not within tolerance for g

Error:
 g3calc=(-37.5434180371816 - 16.1872969107516j) not within tolerance for g3_num=(37.5434180371849 + 16.1872969107509j)

g2 (-79.9267185875938 + 78.3244014859946j)
g3 (37.5434180371849 + 16.1872969107509j)
sign_flipped False
Error:
 g3calc=(-99.0199093760981 - 9.47886025090659j) not within tolerance for g3_num=(99.0199093760982 + 9.47886025090667j)

g2 (-1.34336756843945 + 20.0259178253818j)
g3 (99.0199093760982 + 9.47886025090667j)
sign_flipped False
Error:
 g3calc=(67.7817638239111 - 46.683029492375j) not within tolerance for g3_num=(-67.7817638239115 + 46.6830294923744j)

g2 (-44.9814540863059 - 20.984268857259j)
g3 (-67.7817638239115 + 46.6830294923744j)
sign_flipped False
Error:
 g3calc=(44.3630970181899 + 63.4370618288908j) not within tolerance for g3_num=(-44.3630970181883 - 63.4370618288904j)

g2 (81.7934481335682 + 60.1966871817103j)
g3 (-44.3630970181883 - 63.4370618288904j)
sign_flipped False
Error:
 g3calc=(-82.6062322907027 - 51.9759275444296j) not within tolerance f

Error:
 g3calc=(49.8604328082397 - 99.8075666528946j) not within tolerance for g3_num=(-49.8604328082397 + 99.8075666528969j)

g2 (84.0342491786115 + 2.61129070451034j)
g3 (-49.8604328082397 + 99.8075666528969j)
sign_flipped False
Error:
 g3calc=(64.1359630184211 + 84.5066413995555j) not within tolerance for g3_num=(-64.1359630184217 - 84.5066413995557j)

g2 (-41.1566104236598 - 8.80929130787622j)
g3 (-64.1359630184217 - 84.5066413995557j)
sign_flipped False
Error:
 g3calc=(81.9020980064891 + 88.9717504700151j) not within tolerance for g3_num=(-81.9020980064906 - 88.9717504700147j)

g2 (-86.9733094148034 + 11.3651791149227j)
g3 (-81.9020980064906 - 88.9717504700147j)
sign_flipped False
Error:
 g3calc=(52.9627014259418 + 38.8545615633033j) not within tolerance for g3_num=(-52.9627014259441 - 38.8545615633015j)

g2 (68.2267620066617 + 76.9698742708144j)
g3 (-52.9627014259441 - 38.8545615633015j)
sign_flipped False
Error:
 g3calc=(78.8461375410729 - 23.0565645385294j) not within tolerance

Error:
 g3calc=(-48.8878041665304 + 97.7774917846576j) not within tolerance for g3_num=(48.8878041665322 - 97.7774917846566j)

g2 (-93.3921841709261 - 22.7029518507478j)
g3 (48.8878041665322 - 97.7774917846566j)
sign_flipped False
Error:
 g3calc=(63.5855898316239 + 72.2236849815277j) not within tolerance for g3_num=(-63.5855898316248 - 72.2236849815288j)

g2 (82.0235459070888 - 35.1030616841102j)
g3 (-63.5855898316248 - 72.2236849815288j)
sign_flipped False
Error:
 g3calc=(-49.8478927678906 - 24.502829848781j) not within tolerance for g3_num=(49.8478927678908 + 24.5028298487808j)

g2 (2.76602721225336 - 34.7167175549006j)
g3 (49.8478927678908 + 24.5028298487808j)
sign_flipped False
Error:
 g3calc=(-7.6000120925967 - 12.5125512961501j) not within tolerance for g3_num=(7.60001209259671 + 12.5125512961508j)

g2 (1.91861281214856 - 49.20168913975j)
g3 (7.60001209259671 + 12.5125512961508j)
sign_flipped False
Error:
 g3calc=(-30.6307935004071 + 60.2313368006038j) not within tolerance for g3

...

Error:
 g3calc=(-27.4567141999261 + 26.2221353142639j) not within tolerance for g3_num=(27.4567141999264 - 26.2221353142634j)

g2 (-99.7628227589862 + 90.7968007963822j)
g3 (27.4567141999264 - 26.2221353142634j)
sign_flipped False
Error:
 g3calc=(38.6861284323666 - 62.681420281992j) not within tolerance for g3_num=(-38.6861284323672 + 62.6814202819928j)

g2 (22.2982786603693 - 81.9578956989334j)
g3 (-38.6861284323672 + 62.6814202819928j)
sign_flipped False
Error:
 g3calc=(-19.9290972919789 + 18.6561526335135j) not within tolerance for g3_num=(19.9290972919795 - 18.6561526335149j)

g2 (-13.631566064403 - 72.7933979219974j)
g3 (19.9290972919795 - 18.6561526335149j)
sign_flipped False
Error:
 g3calc=(-92.1889482586444 + 87.9638366846944j) not within tolerance for g3_num=(92.1889482586451 - 87.9638366846952j)

g2 (-61.5555658552721 + 37.2766955052635j)
g3 (92.1889482586451 - 87.9638366846952j)
sign_flipped False
Error:
 g3calc=(-30.6929170384791 + 63.6144763594662j) not within tolerance fo

g3 (99.6430356317638 - 57.562085010389j)
sign_flipped False
Error:
 g3calc=(34.3543983755819 + 45.7961255292225j) not within tolerance for g3_num=(-34.3543983755813 - 45.7961255292238j)

g2 (65.5299020945046 + 20.2885527776927j)
g3 (-34.3543983755813 - 45.7961255292238j)
sign_flipped False
Error:
 g3calc=(-22.5095034386806 - 38.8691733497438j) not within tolerance for g3_num=(22.509503438682 + 38.8691733497438j)

g2 (14.2918294254909 - 55.0819941282286j)
g3 (22.509503438682 + 38.8691733497438j)
sign_flipped False
Error:
 g3calc=(0.404287232638364 + 66.3257734146653j) not within tolerance for g3_num=(-0.404287232635769 - 66.3257734146655j)

g2 (54.6554071800619 + 72.480886844267j)
g3 (-0.404287232635769 - 66.3257734146655j)
sign_flipped False
Error:
 g3calc=(22.0228801266881 - 12.5620124437429j) not within tolerance for g3_num=(-22.0228801266895 + 12.5620124437431j)

g2 (17.9469076337911 - 57.03725951845j)
g3 (-22.0228801266895 + 12.5620124437431j)
sign_flipped False
Error:
 g3calc=(12.

Error:
 g3calc=(70.8553174943117 - 43.7244349612067j) not within tolerance for g3_num=(-70.855317494313 + 43.7244349612069j)

g2 (-92.8084513937357 + 17.443258016058j)
g3 (-70.855317494313 + 43.7244349612069j)
sign_flipped False
Error:
 g3calc=(-59.1550592685529 + 4.89896851099027j) not within tolerance for g3_num=(59.1550592685532 - 4.89896851098972j)

g2 (-41.331578930723 - 26.4703690214957j)
g3 (59.1550592685532 - 4.89896851098972j)
sign_flipped False
Error:
 g3calc=(52.9480440107763 - 82.4384814467963j) not within tolerance for g3_num=(-52.9480440107763 + 82.4384814467963j)

g2 (-29.0042778461098 - 26.1464556965681j)
g3 (-52.9480440107763 + 82.4384814467963j)
sign_flipped False
Error:
 g3calc=(-54.4070596154356 + 50.9238015657878j) not within tolerance for g3_num=(54.4070596154362 - 50.9238015657881j)

g2 (36.0360055263598 + 34.4469122764296j)
g3 (54.4070596154362 - 50.9238015657881j)
sign_flipped False
Error:
 g3calc=(-73.5597932947097 - 71.3105399082613j) not within tolerance for

Error:
 g3calc=(-46.6666036585149 + 74.0530252893235j) not within tolerance for g3_num=(46.6666036585145 - 74.0530252893267j)

g2 (-91.7047749607569 - 56.7954662924708j)
g3 (46.6666036585145 - 74.0530252893267j)
sign_flipped False
Error:
 g3calc=(69.0912355339594 - 69.4428018788841j) not within tolerance for g3_num=(-69.0912355339595 + 69.4428018788841j)

g2 (20.5958488961587 + 4.58633185281336j)
g3 (-69.0912355339595 + 69.4428018788841j)
sign_flipped False
Error:
 g3calc=(-57.1365880819744 - 51.1623394029087j) not within tolerance for g3_num=(57.1365880819749 + 51.1623394029101j)

g2 (-50.4093566312687 + 83.8348292199103j)
g3 (57.1365880819749 + 51.1623394029101j)
sign_flipped False
Error:
 g3calc=(-9.98673352748632 + 81.7707045829249j) not within tolerance for g3_num=(9.98673352748643 - 81.7707045829248j)

g2 (15.1098910101011 + 30.9759116667452j)
g3 (9.98673352748643 - 81.7707045829248j)
sign_flipped False
Error:
 g3calc=(31.1960440752904 - 22.093579413046j) not within tolerance for

g2 (0.705599978162352 - 92.1816038092067j)
g3 (-46.3988187463389 + 97.4240322809396j)
sign_flipped False
Error:
 g3calc=(37.6800759538569 + 35.7765357032835j) not within tolerance for g3_num=(-37.6800759538562 - 35.7765357032847j)

g2 (60.8937443189212 + 15.1236549850694j)
g3 (-37.6800759538562 - 35.7765357032847j)
sign_flipped False
Error:
 g3calc=(35.7766789492211 - 9.61945890318507j) not within tolerance for g3_num=(-35.7766789492228 + 9.61945890318523j)

g2 (-62.6135380186679 + 6.51369536488129j)
g3 (-35.7766789492228 + 9.61945890318523j)
sign_flipped False
Error:
 g3calc=(-27.292639852216 - 86.5089967351074j) not within tolerance for g3_num=(27.2926398522153 + 86.5089967351105j)

g2 (-60.5530285805738 + 92.3516087380135j)
g3 (27.2926398522153 + 86.5089967351105j)
sign_flipped False
Error:
 g3calc=(-52.5001650015624 + 10.6396142313272j) not within tolerance for g3_num=(52.5001650015653 - 10.6396142313269j)

g2 (-86.9679216884264 - 55.5335317173489j)
g3 (52.5001650015653 - 10.639614

In [36]:
# Using the modified package
test_accuracy_of_g_from_omega_and_omega_from_g(
    wst.omega_from_g,
    wst.g_from_omega
)

Error:
 g2calc=(-84.9838077468117 + 67.7740184042109j) not within tolerance for g2_num=(-84.9838077512492 + 67.7740183766262j)
g3calc=(-1.36840975862575 + 0.715088216885373j) not within tolerance for g3_num=(-1.36840975861301 + 0.71508821648878j)

0.001


## 2. Calculating the roots e in terms of theta functions

- The roots e1, e2, e3 can be expressed in terms of jacobi theta functions: https://dlmf.nist.gov/23.6
- If the half-periods are known or already calculated, it is approximately 10x faster to calculate the roots e1, e2, e3 using known formulas in terms of theta functions than to numerically solve the cubic 4 * z**3 - g2 * z - g3 = 0
- Note: It seems the original package has now switched to using theta functions for the roots

In [80]:
g2_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
g3_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')

# Calculate the half-periods from g2 and g3
omegas = wst.omega_from_g(g2_num, g3_num)
omega1 = omegas[0]
omega2 = omegas[1]

# Swap the sign of omega2 if needed to get an allowed value for tau = omega2/omega1
if im(omega2/omega1) <= 0:
    omega2 = -omega2

# Obtain the roots e1, e2, e3 using omega1 and omega2 with theta functions in the modified package    
e1_, e2_, e3_ = wst.roots_from_omega1_omega2(omega1, omega2)

# Obtain the roots e1, e2, e3 using g2 and g3, solve the cubic with the original package    
e1, e2, e3 = polyroots([4, 0, -g2_num, -g3_num])

print(e1_, e2_, e3_)
print(e1, e2, e3)

(4.21862608420373 - 0.541419183621149j) (-0.101791599340047 + 1.40040687435132j) (-4.11683448486369 - 0.858987690730171j)
(4.21862608420371 - 0.541419183621149j) (-4.11683448486366 - 0.858987690730169j) (-0.101791599340047 + 1.40040687435132j)


In [73]:
Ntests = 100
time_roots_from_omega = 0
time_roots_from_g = 0

for _ in range(Ntests):
    try:
        
        # Define random values for g2 and g3 
        # Picks values from within the complex square -num_span ... num_span, -1i*num_span ... 1i*num_span
        g2_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
        g3_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
        
        # Calculate the half-periods from g2 and g3
        omegas = wst.omega_from_g(g2_num, g3_num)
        omega1 = omegas[0]
        omega2 = omegas[1]

        # Swap the sign of omega2 if needed to get an allowed value for tau = omega2/omega1
        if im(omega2/omega1) <= 0:
            omega2 = -omega2
            
        # Obtain the roots e1, e2, e3 using omega1 and omega2 with theta functions and time it    
        time_roots_from_omega += timing(wst.roots_from_omega1_omega2, omega1, omega2)
        
        # Obtain the roots e1, e2, e3 using g2 and g3, solve the cubic and time it    
        time_roots_from_g += timing(wst.sorted_roots_from_g2_g3, g2_num, g3_num)
        
    except Exception as e:
        print(e)

# Calculate average times
average_time_roots_from_omega = time_roots_from_omega / Ntests
average_time_roots_from_g = time_roots_from_g / Ntests

print("average_time_roots_from_omega = ", average_time_roots_from_omega)
print("average_time_roots_from_g = ", average_time_roots_from_g)
print("average_time_roots_from_omega/average_time_roots_from_g = ", 
      average_time_roots_from_omega/average_time_roots_from_g)

average_time_roots_from_omega =  0.00024413072098104747
average_time_roots_from_g =  0.0015215324469863838
average_time_roots_from_omega/average_time_roots_from_g =  0.1604505519843391


## 3. Picking the correct option from a choice of two values when inverting the Weierstrass P function

- The Weierstrass Elliptic function is a meromorphic function of order 2 and thus there are two non-congruent solutions to the equation: wp(z) = c
- This means care must be taken to pick the right one when attempting to invert the Weierstrass function.
- If the value of the derivative, i.e. the value of Weirstrass P prime wpp at z, is known, this can be used to pick from the two possible choices of z.
- As the Weierstrass P function is even, if wp(z) = c then wp(-z) = c, but as Weierstrass P Prime is odd wpp(-z) = -wpp(z) hence if we know wpp(z) we can pick the right sign for z. 

## 4. Testing the accuracy of the inverse Weierstrass P function

In [31]:
def test_accuracy_of_inverse_weierstrass(
    omega_from_g_function,
    w_from_z_and_omega_function,
    inv_w_from_omega_and_w_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define a random value for z
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            z = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Define random values for g2 and g3 
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            num_span = 100
            g2_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega2, omega1 = (omegas[0], omegas[1])

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
            omega = (omega1, omega2)

            # Calculate the Weierstrass P function at z
            w_num = w_from_z_and_omega_function(z, omega)

            # Calculate the value for z 
            # by evaluating the inverse Weierstrass P function 
            # at the corresponding value of Weierstrass P, w_num.
            # This should be equal to the starting value for z 
            # modulo lattice periods
            z_calc = inv_w_from_omega_and_w_function(w_num, omega)

            # Calculate the Weierstrass P function at z_calc
            # By comparing the values for Weierstrass P 
            # we can test the inversion function without dealing with
            # modulo lattice periods
            w_z_calc = w_from_z_and_omega_function(z_calc, omega)

            # Evaluate the difference between the Weierstrass P function
            # evaluated at the original z and the z obtained from the inverse Weierstrass
            if not almosteq(w_num, w_z_calc, tolerance):
                raise Exception(
                    '''
                    The value for Weierstrass P function evaluated at the original z value
                    and at the value of z calculated from the inverse Weierstrass function differ
                    by more than the tolerance.'
                    '''
                )

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
                print('w_num', w_num)
                print('w_z_calc', w_z_calc)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [25]:
# Original package
test_accuracy_of_inverse_weierstrass(
    omega_from_g,
    wp,
    invwp,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


In [26]:
# Modified package
test_accuracy_of_inverse_weierstrass(
    wst.omega_from_g,
    wst.wp,
    wst.invwp,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


# 5. Testing the accuracy of Weierstrass P Prime (Weierstrass P derivative) 

In [47]:
def test_accuracy_of_weierstrass_p_prime(
    omega_from_g_function,
    w_from_z_and_omega_function,
    w_prime_from_z_and_omega_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):
    """
    Note: this test does not check the sign of Weierstrass P Prime as its value is squared
    """

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define a random value for z
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            z = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Define random values for g2 and g3 
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            num_span = 100
            g2_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega2, omega1 = (omegas[0], omegas[1])

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
            omega = (omega1, omega2)

            # Calculate the Weierstrass P function at z
            # and the value of the corresponding cubic
            w_num = w_from_z_and_omega_function(z, omega)
            w_prime_cubic = 4 * w_num ** 3 - g2_num * w_num - g3_num

            # Calculate the Weierstrass P Prime function at z and its square
            w_prime = w_prime_from_z_and_omega_function(z, omega)
            w_prime_sqrd = w_prime ** 2

            # Evaluate the difference between the Weierstrass P function in the cubic
            # and the Weierstrass P Prime function squared
            if not almosteq(w_prime_cubic, w_prime_sqrd, tolerance):
                raise Exception(
                    '''
                    The value for Weierstrass P function evaluated in the cubic
                    and the value for Weierstrass P Prime squared differ
                    by more than the tolerance.'
                    '''
                )

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
                print('w_prime_cubic', w_prime_cubic)
                print('w_prime_sqrd', w_prime_sqrd)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [59]:
# Original package
test_accuracy_of_weierstrass_p_prime(
    wst.omega_from_g, # Using the modified version for omega as the original package has an error
    wp,
    wpprime,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


In [58]:
# Modified package
test_accuracy_of_weierstrass_p_prime(
    wst.omega_from_g,
    wst.wp,
    wst.wpprime,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0
